In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("dark_background")
sns.set_palette("dark")
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score,accuracy_score,recall_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools
import seaborn as sns

In [2]:
df = pd.read_csv("D:\\Portfolio\\03_IMDB_reviews\\IMDB_Preprocessed.csv")
df.head()

,Unnamed: 0,sentiment,text_clean
0,0,1,one reviewers mentioned watching 1 oz episode ...
1,1,1,wonderful little production br br filming tech...
2,2,1,thought wonderful way spend time hot summer we...
3,3,0,basically family little boy jake thinks zombie...
4,4,1,petter mattei love time money visually stunnin...


In [3]:
df.shape

(49582, 3)

In [4]:
df["sentiment"].value_counts()

1    24884
0    24698
Name: sentiment, dtype: int64

In [5]:
y = df['sentiment'].values
X = df[['text_clean']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)

In [6]:
df.loc[df['text_clean'].isnull()].shape

(0, 3)

In [ ]:
# Modeling with Word2Vec

In [7]:
from gensim.models import Word2Vec

i = 0
list_of_sentance = []
for sentence in df['text_clean'].astype(str):
    list_of_sentance.append(sentence.split())

w2v_model = Word2Vec(list_of_sentance, min_count=1, vector_size=300, workers=4)
w2v_words = list(w2v_model.wv.key_to_index.keys())
list_of_sentance[:3]

[['one',
  'reviewers',
  'mentioned',
  'watching',
  '1',
  'oz',
  'episode',
  'hooked',
  'right',
  'exactly',
  'happened',
  'br',
  'br',
  'first',
  'thing',
  'struck',
  'oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'go',
  'trust',
  'not',
  'show',
  'faint',
  'hearted',
  'timid',
  'show',
  'pulls',
  'no',
  'punches',
  'regards',
  'drugs',
  'sex',
  'violence',
  'hardcore',
  'classic',
  'use',
  'word',
  'br',
  'br',
  'called',
  'oz',
  'nickname',
  'given',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary',
  'focuses',
  'mainly',
  'emerald',
  'city',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'face',
  'inwards',
  'privacy',
  'not',
  'high',
  'agenda',
  'em',
  'city',
  'home',
  'many',
  'aryans',
  'muslims',
  'gangstas',
  'latinos',
  'christians',
  'italians',
  'irish',
  'scuffles',
  'death',
  'stares',
  'dodgy',
  'dealings',
  

In [8]:
from tqdm import tqdm 
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each sentence
    sent_vec = np.zeros(300)
    cnt_words =0; # num of words with a valid vector in the sentence
    for word in sent: # for each word in a sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|████████████████████████████████████████████████████████████████████████████| 49582/49582 [27:48<00:00, 29.71it/s]

49582
300


In [9]:
X=np.array(sent_vectors)
y = df['sentiment'].values
print("x shape=",X.shape,"  y shape=",y.shape)

x shape= (49582, 300)   y shape= (49582,)


In [10]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)

# Gausian Naive Bayes

In [11]:
# Training Naive bayes
# Finding Accuracy, AUC, False positive rate, True positive rate, confusion matrix and classificatio report
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
accNB = accuracy_score(y_test, pred)
y_pred_prob = NB.predict_proba(X_test)
predT=NB.predict(X_train)
aucScoreNB = roc_auc_score(y_test,  y_pred_prob[:,1])
fprNB, tprNB, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("***AUC score***")
print("AUC score for NB is ",aucScoreNB)
print("***Accuracy score***")
print("Train Accuracy score for NB is ",accuracy_score(y_train, predT))
print("Test Accuracy score for NB is ",accuracy_score(y_test, pred))
print("***Recall score***")
print("Train recall score for NB is ",recall_score(y_train, predT))
print("Test recall score for NB is ",recall_score(y_test, pred))

[[5661 1749]
 [1739 5726]]
              precision    recall  f1-score   support

           0       0.77      0.76      0.76      7410
           1       0.77      0.77      0.77      7465

    accuracy                           0.77     14875
   macro avg       0.77      0.77      0.77     14875
weighted avg       0.77      0.77      0.77     14875

***AUC score***
AUC score for NB is  0.8430074671453739
***Accuracy score***
Train Accuracy score for NB is  0.7639957357305442
Test Accuracy score for NB is  0.7655126050420168
***Recall score***
Train recall score for NB is  0.7643377920661347
Test recall score for NB is  0.7670462156731414


# Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
# Create a Logistic Regression classifier
LR = LogisticRegression()
LR.fit(X_train, y_train)
pred = LR.predict(X_test)
accLR = accuracy_score(y_test, pred)
y_pred_prob = LR.predict_proba(X_test)
predT=LR.predict(X_train)
aucScoreLR = roc_auc_score(y_test,  y_pred_prob[:,1])
fprLR, tprLR, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("***AUC score***")
print("AUC score for LR is ",aucScoreLR)
print("***Accuracy score***")
print("Train Accuracy score for LR is ",accuracy_score(y_train, predT))
print("Test Accuracy score for LR is ",accuracy_score(y_test, pred))
print("***Recall score***")
print("Train recall score for LR is ",recall_score(y_train, predT))
print("Test recall score for LR is ",recall_score(y_test, pred))

[[6460  950]
 [ 907 6558]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      7410
           1       0.87      0.88      0.88      7465

    accuracy                           0.88     14875
   macro avg       0.88      0.88      0.88     14875
weighted avg       0.88      0.88      0.88     14875

***AUC score***
AUC score for LR is  0.9434809497854586
***Accuracy score***
Train Accuracy score for LR is  0.873887112109949
Test Accuracy score for LR is  0.8751596638655462
***Recall score***
Train recall score for LR is  0.8801308915551983
Test recall score for LR is  0.8784996651038178


# Decision tree

In [13]:
# Decision Trees (Multiple if-else statements!)
from sklearn.tree import DecisionTreeClassifier
# Create a DecisionTree classifier
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)
pred = DT.predict(X_test)
accDT = accuracy_score(y_test, pred)
y_pred_prob = DT.predict_proba(X_test)
predT=DT.predict(X_train)
aucScoreDT = roc_auc_score(y_test,  y_pred_prob[:,1])
fprDT, tprDT, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("***AUC score***")
print("AUC score for DT is ",aucScoreDT)
print("***Accuracy score***")
print("Train Accuracy score for DT is ",accuracy_score(y_train, predT))
print("Test Accuracy score for DT is ",accuracy_score(y_test, pred))
print("***Recall score***")
print("Train recall score for DT is ",recall_score(y_train, predT))
print("Test recall score for DT is ",recall_score(y_test, pred))

[[5458 1952]
 [1941 5524]]
              precision    recall  f1-score   support

           0       0.74      0.74      0.74      7410
           1       0.74      0.74      0.74      7465

    accuracy                           0.74     14875
   macro avg       0.74      0.74      0.74     14875
weighted avg       0.74      0.74      0.74     14875

***AUC score***
AUC score for DT is  0.7382794019414036
***Accuracy score***
Train Accuracy score for DT is  1.0
Test Accuracy score for DT is  0.7382857142857143
***Recall score***
Train recall score for DT is  1.0
Test recall score for DT is  0.7399866041527127


# Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
# Create a RandomForestClassifier classifier
RF = RandomForestClassifier()
RF.fit(X_train, y_train)
pred = RF.predict(X_test)
accRF = accuracy_score(y_test, pred)
y_pred_prob = RF.predict_proba(X_test)
predT=RF.predict(X_train)
aucScoreRF = roc_auc_score(y_test,  y_pred_prob[:,1])
fprRF, tprRF, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("***AUC score***")
print("AUC score for RF is ",aucScoreRF)
print("***Accuracy score***")
print("Train Accuracy score for RF is ",accuracy_score(y_train, predT))
print("Test Accuracy score for RF is ",accuracy_score(y_test, pred))
print("***Recall score***")
print("Train recall score for RF is ",recall_score(y_train, predT))
print("Test recall score for RF is ",recall_score(y_test, pred))

[[6154 1256]
 [1069 6396]]
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      7410
           1       0.84      0.86      0.85      7465

    accuracy                           0.84     14875
   macro avg       0.84      0.84      0.84     14875
weighted avg       0.84      0.84      0.84     14875

***AUC score***
AUC score for RF is  0.9197318118832555
***Accuracy score***
Train Accuracy score for RF is  1.0
Test Accuracy score for RF is  0.8436974789915966
***Recall score***
Train recall score for RF is  1.0
Test recall score for RF is  0.8567983924983256


# XG Boost

In [15]:
from xgboost import XGBClassifier
# Create a RandomForestClassifier classifier
XGB = XGBClassifier()
XGB.fit(X_train, y_train)
pred = XGB.predict(X_test)
accXGB = accuracy_score(y_test, pred)
y_pred_prob = XGB.predict_proba(X_test)
predT=XGB.predict(X_train)
aucScoreXGB = roc_auc_score(y_test,  y_pred_prob[:,1])
fprXGB, tprXGB, thresholds = roc_curve(y_test, y_pred_prob[:,1] )
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
print("***AUC score***")
print("AUC score for XGB is ",aucScoreXGB)
print("***Accuracy score***")
print("Train Accuracy score for XGB is ",accuracy_score(y_train, predT))
print("Test Accuracy score for XGB is ",accuracy_score(y_test, pred))
print("***Recall score***")
print("Train recall score for XGB is ",recall_score(y_train, predT))
print("Test recall score for XGB is ",recall_score(y_test, pred))

[[6339 1071]
 [ 989 6476]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.86      7410
           1       0.86      0.87      0.86      7465

    accuracy                           0.86     14875
   macro avg       0.86      0.86      0.86     14875
weighted avg       0.86      0.86      0.86     14875

***AUC score***
AUC score for XGB is  0.9354951266052195
***Accuracy score***
Train Accuracy score for XGB is  0.9902325179358631
Test Accuracy score for XGB is  0.8615126050420168
***Recall score***
Train recall score for XGB is  0.9892645961306619
Test recall score for XGB is  0.8675150703281983


Logistic regression seems to be best by use of Word to Vec vectorization